In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")
import helpers
import glm
import math

%load_ext autoreload
%autoreload 2

# Material matching
In the previous weeks you learned how a depth map can be reconstructed from a set of images. To create a complete reconstruction of the physical world we need to capture not only shape, but also appearance.

Capturing the surface appearance of an object might seem simple: just get the pixel values of the images used for reconstruction. However, this does not work when viewing the virtual object from different angles. Furthermore, we would capture the scenes lighting with no way to change it. Instead we want to capture the way the surface reacts to light.

Visualizing objects with different materials has been an active research topic in computer graphics since its inception. We use mathematical models (Bidirectional Reflection Distribution Function, or BRDF for short) to model how light reflects of a surface. By changing the parameters of these models many different materials can be represented. The goal of material matching is to match a mathematical model to real world appearance by finding these parameters.

## Input Images
Usually material matching would come at the end of the multi-view stereo pipeline. But for educational purposes we have chosen to generate input data using Blender. The input consists of three different images as shown below.

The first image contains a picture of the object under known lighting conditions. The second image stores for each pixel a 3D position in space: the 3D location of the surface of the bunny at that pixel. Finally, the third image stores for each pixel a normal vector: the orientation of the surface. Examples of these three images are shown below.

In [ ]:
camera_pos = glm.vec3(2, -2, 1.5)
light_pos = glm.vec3(1, -2.0, 3.5)
light_color = glm.vec3(0.8, 0.8, 0.8)

colors = np.load(os.path.join(helpers.dataset_folder, "week5", "shaded_bunny.npy"))
positions = helpers.imread_hdr(os.path.join(helpers.dataset_folder, "week5", "position0001.exr"), 0.25, nn_interpolation=True)
normals = helpers.imread_hdr(os.path.join(helpers.dataset_folder, "week5", "normal0001.exr"), 0.25, nn_interpolation=True)

helpers.show_images({
    "Color (Phong Model)": np.clip(colors, 0, 1),
    "3D Position": positions,
    "Surface Normal": normals / 2 + 0.5
}, nrows=1, ncols=3)

### Exercise 4 (2 points)
Your job is to find a material which matches the appearance of the bunny in the given image. Find the parameters $k_s$ and $k_d$ of the Phong model that were used to render the images by formulating the problem as a linear system and finding the least squares solution. Each pixel that shows the bunny (= not background pixels) should add contraints to the linear system.

$t$ is assumed to already be known and for this exercise is guaranteed to be correct. The light is provided so that you can compute the incoming light- direction&color at any point.

Construct a linear system to find $k_d$ and $k_s$ and solve it using `np.linalg.lstsq(A, b)[0]`.

In [ ]:
def solve_phong_kd_ks(t, light_color, light_position, observer_position, pixels):
    kd = np.array([0, 0, 0])
    ks = np.array([0, 0, 0])
    # TODO: Find the values of kd and ks by constructing and solving a linear system.
    for surface_normal, surface_position, observed_color in pixels:
        # surface_normal gives the surface normal seen in this pixel
        # surface_position gives the position in 3D space seen in this pixel
        # observed_color gives the color of this pixel
        pass
    # YOUR CODE HERE
    return kd, ks

# Get surface normal, surface position and observed color for each non-background pixel.
pixels = zip(normals.reshape(-1, 3), positions.reshape(-1, 3), colors.reshape(-1, 3))
pixels = [pixel for pixel in pixels if glm.dot(pixel[0], pixel[0]) > 0]
# Filter out edge cases where the light is behind the surface of the object.
pixels = [(n, s, c) for n, s, c in pixels if np.dot(n, light_pos - s) > 0.01]

import warnings
with warnings.catch_warnings():
    # Ignore warnings about "FutureWarning: `rcond` parameter will change to the default ..." in np.linalg.lstsq
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    kd_estimate, ks_estimate = solve_phong_kd_ks(5, light_color, light_pos, camera_pos, pixels)

print(f"Your estimated parameters: kd={kd_estimate}, ks={ks_estimate}")
print(f"Actual parameters: kd=[0.7, 0.4, 0.9], ks=[0.3, 0.2, 0.4]\n")

### Tests of exercise 4
The parameters `kd` and `ks` that your solution returns should be within $0.0001$ of the actual values.

In [ ]:
# Add your own tests here.

In [ ]:
# DO NOT REMOVE, MODIFY, OR COPY THIS CELL


## Computing shininess
The shininess term $t$ does not have a linear relation to the output. This problem is very common in more complex reflection models which are never just a linear blend of parameters.

$$
k_d I (L \cdot N) + k_s I (E \cdot R)^t
$$

Assume that we know $k_d$ and $k_s$, making $t$ the only unknown variable. We can reformulate the Phong formula (above) such that it states $t=\text{...}$ . With this we can estimate $t$ for every pixel & color channel. Take the average over all (non background) pixels to get an accurate estimate of the shininess value $t$ of the sphere.

### Exercise 5 (2 points)
Derive the formula for $t$ given $k_d$ and $k_s$ and use it to compute the mean estimated t value for all pixels. You may assume that the provided values of $k_d$ and $k_s$ exactly match the provided images.

**Tip**: There are many pixels which have an extremely low specular contribution. Trying to estimate $t$ from these pixels will lead to mathematical issues (e.g. division by zero or logarithm of zero) or numerical precision limitations (division by very small number). We recommend skipping these pixels as they do not contribute to an accurate estimate of $t$.

In [ ]:
def solve_phong_t(kd, ks, light_color, light_position, observer_position, pixels):
    t = 12.3
    # TODO: Find the value for t which is the same for all pixels. Be aware of numerical issues.
    for surface_normal, surface_position, observed_color in pixels:
        # surface_normal gives the surface normal seen in this pixel
        # surface_position gives the position in 3D space seen in this pixel
        # observed_color gives the color of this pixel
         pass
    # YOUR CODE HERE
    return t

# Get surface normal, surface position and observed color for each non-background pixel.
pixels = zip(normals.reshape(-1, 3), positions.reshape(-1, 3), colors.reshape(-1, 3))
pixels = [pixel for pixel in pixels if glm.dot(pixel[0], pixel[0]) > 0]
# Filter out edge cases where the light is behind the surface of the object.
pixels = [(n, s, c) for n, s, c in pixels if np.dot(n, light_pos - s) > 0.01]
t_estimate = solve_phong_t(glm.vec3(0.7, 0.4, 0.9), glm.vec3(0.3, 0.2, 0.4), light_color, light_pos, camera_pos, pixels)

print(f"Your estimated parameters: t={t_estimate}")
print(f"Actual parameters: t=5")

### Tests of exercise 5
Your method should be able to detect the $t$ value with high accuracy ($<0.001$)

In [ ]:
# Add your own tests here.

In [ ]:
# DO NOT REMOVE, MODIFY, OR COPY THIS CELL
